In [25]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import layers
import os

In [27]:
import sklearn

face_dataset = sklearn.datasets.fetch_lfw_people(min_faces_per_person=10)
IMAGE_SIZE = face_dataset.images[0].shape

In [59]:
face_dataset.target.shape

(4324,)

In [122]:
ds = tf.data.Dataset.from_tensor_slices((np.reshape(face_dataset.images, (-1, IMAGE_SIZE[0], IMAGE_SIZE[1], 1)), face_dataset.target))
batch_size = 64
ds = ds.shuffle(len(face_dataset.images), seed=1).batch(batch_size)

In [141]:
TRAIN_SIZE = int(len(face_dataset.images) / batch_size * 0.8)
train_ds = ds.take(TRAIN_SIZE)
test_ds = ds.skip(TRAIN_SIZE)

In [143]:
# Xception model from https://keras.io/examples/vision/image_classification_from_scratch/#using-image-data-augmentation

def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=IMAGE_SIZE + (1, ), num_classes=face_dataset.target_names.shape[0])

In [144]:
for imgs, labels in test_ds.take(1):
    print(imgs.shape)

(64, 62, 47, 1)


In [145]:
epochs = 25

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]
model.compile(
    optimizer=keras.optimizers.legacy.Adam(1e-3), # legacy for m1 support
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=test_ds,
)

Epoch 1/25


54/54 [==============================] - 62s 1s/step - loss: 4.4701 - accuracy: 0.1400 - val_loss: 5.0095 - val_accuracy: 0.1348
Epoch 2/25
54/54 [==============================] - 67s 1s/step - loss: 3.4353 - accuracy: 0.2766 - val_loss: 5.0256 - val_accuracy: 0.0081
Epoch 3/25
54/54 [==============================] - 69s 1s/step - loss: 2.2077 - accuracy: 0.4789 - val_loss: 5.0645 - val_accuracy: 0.0058
Epoch 4/25
54/54 [==============================] - 66s 1s/step - loss: 1.3268 - accuracy: 0.6736 - val_loss: 5.1042 - val_accuracy: 0.0104
Epoch 5/25
54/54 [==============================] - 65s 1s/step - loss: 0.7898 - accuracy: 0.8061 - val_loss: 5.3085 - val_accuracy: 0.0138
Epoch 6/25
54/54 [==============================] - 67s 1s/step - loss: 0.4557 - accuracy: 0.8961 - val_loss: 5.5168 - val_accuracy: 0.0092
Epoch 7/25
54/54 [==============================] - 70s 1s/step - loss: 0.2512 - accuracy: 0.9453 - val_loss: 6.0816 - val_accuracy: 0.0058
Epoch 8/25
54/54 [=============